In [1]:
import os
import re
import torch as T
import torch.nn.functional as F
import pandas as pd

In [2]:
import sys
sys.path.append("../scripts/")

In [3]:
from bert4rec_model import RecommendationModel
from bert4rec_dataset import Bert4RecDataset

In [4]:
data = pd.read_csv("../data/ratings_mapped.csv")

In [5]:
data.head()

,userId,movieId,rating,timestamp,movieId_mapped
0,429,595,5.0,828124615,514
1,429,588,5.0,828124615,508
2,429,590,5.0,828124615,510
3,429,592,5.0,828124615,511
4,429,432,3.0,828124615,378


In [6]:
rec_model = RecommendationModel(9725)

NUM POS:  128


In [7]:
from AttentionTransformer.utilities import count_model_parameters

In [8]:
count_model_parameters(rec_model) / 1e6

4.882173

In [9]:
dataset = Bert4RecDataset(data_csv=data,
                          group_by_col="userId",
                          data_col="movieId_mapped")


In [10]:
from torch.utils.data import DataLoader

In [11]:
dataloader = DataLoader(dataset, batch_size = 8, shuffle = True)

In [12]:
batch_1 = next(iter(dataloader))

In [13]:
batch_1.keys()

dict_keys(['source', 'target', 'source_mask', 'target_mask'])

In [31]:
batch_1['source'][0], batch_1['source'].size()

(tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    1,  708,  909,    1,  517,  129,    1,
          794,  698,  947,    1,  787, 1425, 1550, 1023,  907, 1533, 1906, 1926]),
 torch.Size([8, 120]))

In [15]:
batch_1['target'][0]

tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 1591,  708,  909,  693,  517,  129,  951,
         794,  698,  947, 1003,  787, 1425, 1550, 1023,  907, 1533, 1906, 1926])

In [48]:
model_op = rec_model(batch_1["source"])

BATCH: torch.Size([8, 120])
ITEMS: torch.Size([8, 120, 128])
REPEAT BATCH: torch.Size([8, 120])
ENCODED BATCH: torch.Size([8, 120, 128])
PERMUTED BATCH: torch.Size([120, 8, 128])
AFTER LINEAR BATCH: torch.Size([120, 8, 9725])


In [49]:
model_op.size()

torch.Size([120, 8, 9725])

In [50]:
mask = batch_1["source"] == 1

In [51]:
batch_1["target"].size()

torch.Size([8, 120])

In [52]:
_, predicted = model_op.max(2)


In [53]:
predicted.size()

torch.Size([120, 8])

In [55]:
predicted.permute(1, 0).size()

torch.Size([8, 120])

In [22]:
predicted

tensor([[ 217, 3976,  731, 2816, 3362,  629,  992, 8677],
        [1041, 8969, 1041, 2551, 8969, 5241, 2551, 8969],
        [9224, 9628, 4263, 4263, 4263, 4263, 9628, 6748],
        [9227, 3429, 3429, 2008, 1490, 2008, 3429, 4024],
        [3079, 3079, 5137, 4077, 3079, 5137, 3079, 3079],
        [8677, 8677, 8677, 8677, 8677, 8677, 6361, 8677],
        [1984, 6094, 3373, 3759, 1984, 1497, 3303, 1984],
        [9040, 6047, 9142, 5993, 7928, 7928, 2974,  217],
        [5137, 5137, 5137, 5137, 5137, 5137, 5137,  748],
        [ 486, 4707, 7002, 7614, 7614, 7614, 7614, 9040],
        [9676, 9676, 3079, 3079, 3588, 3588, 9676, 3079],
        [5864, 5864, 5864, 1526, 5864,  851, 5864, 5864],
        [5371, 4279, 8102, 5371, 8102, 8102, 8102,  269],
        [5496, 4938, 4938, 5241, 4938, 2151, 2151, 2151],
        [5657, 9628, 5657, 5657, 5657, 5657, 5657, 5657],
        [1352, 1467, 6240, 2342, 9628, 4340, 2733, 1352],
        [4710, 7867, 5496, 4710, 5496, 4710, 4710, 1595],
        [3976,

In [25]:
predicted.size()

torch.Size([120, 8])

In [26]:
mask.permute(1, 0).size()

torch.Size([120, 8])

In [23]:
mask.size()

torch.Size([8, 120])

In [27]:
## accuracy
y_true = T.masked_select(batch_1["target"].permute(1, 0), mask.permute(1, 0))
predicted = T.masked_select(predicted, mask.permute(1, 0))

In [28]:
(y_true == predicted).double().mean()

tensor(0., dtype=torch.float64)

In [32]:
model_op.size()

torch.Size([960, 9725])

In [33]:
model_op = model_op.view(-1, model_op.size(2))

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [56]:
model_op.size()

torch.Size([120, 8, 9725])

In [57]:
batch_1["target"].view(-1).size()

torch.Size([960])

In [62]:
model_op.permute(1, 0, 2).size()

torch.Size([8, 120, 9725])

In [63]:
batch_1["target"].size()

torch.Size([8, 120])

In [68]:
F.cross_entropy(model_op.permute(1, 2, 0), batch_1["target"])

tensor(9.1236, grad_fn=<NllLoss2DBackward>)

In [70]:
## loss
loss = F.cross_entropy(model_op.permute(1, 2, 0), batch_1["target"], reduction="none")
loss = loss * mask
loss.sum() / (mask.sum() + 1e-8)

tensor(9.3915, grad_fn=<DivBackward0>)

In [37]:
type(model_op), type(batch_1["target"]), type(mask)

(torch.Tensor, torch.Tensor, torch.Tensor)

In [38]:
type(loss)

torch.Tensor

In [39]:
(T.tensor([1, 2, 3]) == T.tensor([1, 0, 3])).double().mean()

tensor(0.6667, dtype=torch.float64)

In [40]:
type(rec_model)

bert4rec_model.RecommendationModel

In [41]:
type(dataloader)

torch.utils.data.dataloader.DataLoader

In [42]:
means = [1, 2, 3.3, 4.5, 6.6, 7, 8]
batch_sizes = [8, 8, 8, 8, 8, 8, 4]
accuracy = T.sum(T.tensor(batch_sizes) * T.tensor(means)) / T.sum(
        T.tensor(batch_sizes))

In [43]:
accuracy.item()

4.3692307472229

In [44]:
predicted.permute(0, 1).size()

RuntimeError: number of dims don't match in permute

In [45]:
predicted.size()

torch.Size([94])